In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report

import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from sklearn.pipeline import Pipeline
#from sklearn.metrics.scorer import make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
import numpy as np
from catboost import Pool, CatBoostClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize

In [4]:
data=pd.read_excel('data.xlsx', dtype=object)

In [5]:
data.head(2)

,Unnamed: 0,TARGET,TEXT
0,0,INSULT,скотина! что сказать
1,7,INSULT,заколоть этого плешивого урода что бы крякнул ...


In [6]:
data=data.sample(500)

In [7]:
data.TARGET.value_counts()

INSULT       259
NORMAL       136
THREAT        75
OBSCENITY     30
Name: TARGET, dtype: int64

In [8]:

def norm(txt):
    lst = re.sub('[^А-Яа-яa-zA-Z]+', ' ', str(txt)).lower() #оставляет только А-я, A-z
    lst = lst.split(' ')

    words = []
    
    for word in lst:
        p = morph.parse(word)[0]
        words.append(p.normal_form)

    return ' '.join(words)

In [9]:
%%time
data['TEXT']=data['TEXT'].apply(norm)

Wall time: 1.74 s


In [10]:
data['VIP']=np.random.randint(4,size=data.shape[0]).astype(str)
data['LOL']=np.random.randint(20,size=data.shape[0]).astype(str)
data['ABC']=np.random.randint(10,size=data.shape[0])

In [11]:
data.head(2)

,Unnamed: 0,TARGET,TEXT,VIP,LOL,ABC
59101,7517,NORMAL,девочка чай и кофе разобрать и передать следит...,1,5,0
37822,182019,THREAT,они бы рука оторвать за такой латание,1,11,4


In [12]:
cat_features = ['VIP','LOL']
text_features = ['TEXT']


In [13]:
X=data[['TEXT','VIP', 'LOL', 'ABC']]
y=data.TARGET

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y.values,test_size=0.15)


In [15]:
train_pool = Pool(
    X_train, 
    y_train, 
    cat_features=cat_features, 
    text_features=text_features, 
    feature_names=list(X_train)
)
valid_pool = Pool(
    X_test, 
    y_test,
    cat_features=cat_features, 
    text_features=text_features, 
    feature_names=list(X_train)
)

catboost_params = {
    'iterations': 3000,
    'learning_rate': 0.01,
    'eval_metric': 'Accuracy',
    'task_type': 'GPU',
    'early_stopping_rounds': 2000,
    'use_best_model': True,
    'verbose': 500
}
#clas_weight = [5,4]
model = CatBoostClassifier(**catboost_params)
model.fit(train_pool, eval_set=valid_pool)

0:	learn: 0.5576471	test: 0.6533333	best: 0.6533333 (0)	total: 27.4ms	remaining: 1m 22s
500:	learn: 0.6964706	test: 0.7066667	best: 0.7200000 (312)	total: 4.68s	remaining: 23.3s
1000:	learn: 0.7341176	test: 0.7200000	best: 0.7200000 (312)	total: 9.4s	remaining: 18.8s
1500:	learn: 0.7576471	test: 0.6933333	best: 0.7200000 (312)	total: 14.2s	remaining: 14.2s
2000:	learn: 0.8117647	test: 0.6666667	best: 0.7200000 (312)	total: 18.8s	remaining: 9.38s
bestTest = 0.72
bestIteration = 312
Shrink model to first 313 iterations.


In [16]:
model.predict(X_test)

array([['INSULT'],
       ['NORMAL'],
       ['NORMAL'],
       ['THREAT'],
       ['NORMAL'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['NORMAL'],
       ['INSULT'],
       ['NORMAL'],
       ['NORMAL'],
       ['NORMAL'],
       ['THREAT'],
       ['NORMAL'],
       ['INSULT'],
       ['NORMAL'],
       ['NORMAL'],
       ['INSULT'],
       ['INSULT'],
       ['NORMAL'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['NORMAL'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['NORMAL'],
       ['INSULT'],
       ['INSULT'],
       ['INSULT'],
       ['NORMAL'],
       ['NORMAL'],
       ['INSULT'],
       ['INSULT'],
       ['INS

In [17]:
y_pred = model.predict(X_test)

In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      INSULT       0.74      0.84      0.79        44
      NORMAL       0.67      0.74      0.70        19
   OBSCENITY       0.00      0.00      0.00         3
      THREAT       0.75      0.33      0.46         9

    accuracy                           0.72        75
   macro avg       0.54      0.48      0.49        75
weighted avg       0.69      0.72      0.69        75



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
y_pred = model.predict_proba(X_test)
y_pred

array([[0.60795483, 0.1842885 , 0.07539727, 0.1323594 ],
       [0.24122689, 0.60286084, 0.0636513 , 0.09226097],
       [0.39842586, 0.45245831, 0.04917685, 0.09993898],
       [0.35756469, 0.19966334, 0.08322055, 0.35955143],
       [0.40992073, 0.41304984, 0.05143812, 0.1255913 ],
       [0.46650948, 0.3671678 , 0.06346264, 0.10286009],
       [0.59330223, 0.19312489, 0.07751941, 0.13605347],
       [0.45080125, 0.38701966, 0.06804587, 0.09413322],
       [0.52181535, 0.20766175, 0.08681401, 0.18370889],
       [0.6784937 , 0.13654459, 0.07541287, 0.10954884],
       [0.61943342, 0.18659227, 0.0776638 , 0.1163105 ],
       [0.57845685, 0.15395656, 0.07846657, 0.18912001],
       [0.43570744, 0.39074919, 0.05195838, 0.12158499],
       [0.64321093, 0.14213944, 0.06685001, 0.14779962],
       [0.43348286, 0.38738909, 0.05269365, 0.1264344 ],
       [0.23937692, 0.60795017, 0.06240003, 0.09027288],
       [0.70140211, 0.13047914, 0.06403559, 0.10408316],
       [0.3471045 , 0.46284684,

In [20]:
y_pred_=model.predict(X_test).flatten()
y_pred_

array(['INSULT', 'NORMAL', 'NORMAL', 'THREAT', 'NORMAL', 'INSULT',
       'INSULT', 'INSULT', 'INSULT', 'INSULT', 'INSULT', 'INSULT',
       'INSULT', 'INSULT', 'INSULT', 'NORMAL', 'INSULT', 'NORMAL',
       'NORMAL', 'NORMAL', 'THREAT', 'NORMAL', 'INSULT', 'NORMAL',
       'NORMAL', 'INSULT', 'INSULT', 'NORMAL', 'INSULT', 'INSULT',
       'INSULT', 'INSULT', 'INSULT', 'NORMAL', 'INSULT', 'INSULT',
       'INSULT', 'INSULT', 'INSULT', 'INSULT', 'INSULT', 'INSULT',
       'INSULT', 'INSULT', 'NORMAL', 'INSULT', 'INSULT', 'INSULT',
       'NORMAL', 'NORMAL', 'INSULT', 'INSULT', 'INSULT', 'INSULT',
       'NORMAL', 'INSULT', 'INSULT', 'INSULT', 'INSULT', 'NORMAL',
       'NORMAL', 'NORMAL', 'INSULT', 'INSULT', 'THREAT', 'NORMAL',
       'THREAT', 'INSULT', 'INSULT', 'INSULT', 'NORMAL', 'INSULT',
       'INSULT', 'INSULT', 'INSULT'], dtype=object)

In [21]:
np.max(y_pred,axis=1)

array([0.60795483, 0.60286084, 0.45245831, 0.35955143, 0.41304984,
       0.46650948, 0.59330223, 0.45080125, 0.52181535, 0.6784937 ,
       0.61943342, 0.57845685, 0.43570744, 0.64321093, 0.43348286,
       0.60795017, 0.70140211, 0.46284684, 0.45283932, 0.44171146,
       0.38315394, 0.4477674 , 0.63639063, 0.53825906, 0.50547917,
       0.6710303 , 0.64100798, 0.43701349, 0.48645189, 0.44541092,
       0.69094408, 0.51168486, 0.41928628, 0.61702537, 0.64371951,
       0.5916011 , 0.57679543, 0.6494696 , 0.66067608, 0.50616645,
       0.47921997, 0.50104117, 0.4151869 , 0.56145   , 0.44291796,
       0.59926403, 0.61635074, 0.65799236, 0.43242173, 0.46297004,
       0.65978929, 0.41761487, 0.67412587, 0.55893788, 0.56685417,
       0.42360676, 0.56990581, 0.46776804, 0.6476866 , 0.52380722,
       0.44408061, 0.48544332, 0.48829551, 0.59119741, 0.41019954,
       0.46379594, 0.46297564, 0.5404759 , 0.55806899, 0.61405123,
       0.44479201, 0.58905045, 0.6852849 , 0.52365557, 0.61249

In [22]:
data_test= X_test
data_test['TARGET']=y_test
data_test['proba']=np.max(y_pred,axis=1)
data_test['PRED']=y_pred_
data_test

<ipython-input-22-b72b563b90c7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['TARGET']=y_test
<ipython-input-22-b72b563b90c7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['proba']=np.max(y_pred,axis=1)
<ipython-input-22-b72b563b90c7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

,TEXT,VIP,LOL,ABC,TARGET,proba,PRED
18670,пойти он на хуй пиздобол охуесть,3,9,3,INSULT,0.607955,INSULT
64074,с праздник железнодорожник весь добро и здоро...,3,14,2,NORMAL,0.602861,NORMAL
27154,вырвать всё причендал,0,0,4,THREAT,0.452458,NORMAL
19033,букварь сначала выучить пысара за твой писанин...,1,10,0,THREAT,0.359551,THREAT
36866,рожа мерзопакостный крысиный глаз и вообще это...,2,2,4,INSULT,0.413050,NORMAL
...,...,...,...,...,...,...,...
37670,сейчас через интернет большой шанс что то здет...,2,19,7,INSULT,0.444792,NORMAL
37565,нечего себя один тварь вокруг палцо крутиль че...,3,12,0,INSULT,0.589050,INSULT
25035,я ты сука не только б член отрезать за лугандон,3,1,5,INSULT,0.685285,INSULT
40363,бля это бред посадить нахуй,1,1,5,INSULT,0.523656,INSULT


In [23]:
data_test[(data_test['proba']>0.5) & (data_test['PRED']==data_test['TARGET'])].shape[0]/data_test[(data_test['proba']>0.5) ].shape[0]

0.8181818181818182

In [24]:
data_test[(data_test['proba']>0.6) & (data_test['PRED']==data_test['TARGET'])].shape[0]/data_test[(data_test['proba']>0.6) ].shape[0]

0.9130434782608695

In [25]:
data_test[(data_test['proba']>0.7) & (data_test['PRED']==data_test['TARGET'])].shape[0]/data_test[(data_test['proba']>0.7) ].shape[0]

1.0

In [26]:
data_test[(data_test['proba']>0.8) & (data_test['PRED']==data_test['TARGET'])].shape[0]/data_test[(data_test['proba']>0.8) ].shape[0]

ZeroDivisionError: division by zero

In [27]:
data_test[(data_test['proba']>0.9) & (data_test['PRED']==data_test['TARGET'])].shape[0]/data_test[(data_test['proba']>0.9) ].shape[0]

ZeroDivisionError: division by zero

In [28]:
data_test[(data_test['proba']>0.95) & (data_test['PRED']==data_test['TARGET'])].shape[0]/data_test[(data_test['proba']>0.95) ].shape[0]

ZeroDivisionError: division by zero